# The Neuroscience of Human Movement analysis notebook!

In [ ]:
#Importing libraries


from pathlib import Path

try:
    import numpy as np
except Exception as e:
    print(e)
    %pip install numpy
    import numpy as np

try:
    from scipy.signal import find_peaks
except Exception as e:
    print(e)
    %pip install scipy
    from scipy.signal import find_peaks

try:
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
except Exception as e:
    print(e)
    %pip install plotly
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

In [ ]:
# Insert the path to the data here

freemocap_3d_body_data = np.load("/content/mediapipe_body_3d_xyz.npy")
total_body_com_data = np.load("/content/total_body_center_of_mass_xyz.npy")



In [ ]:
total_body_com_data.shape

### What does the "shape" of the data mean?
The shape of the data is the number of frames by the number of dimensions. In this case, the data is 3D, so the shape is (number of frames, 3).

In [ ]:

mediapipe_indices = ['nose',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_ear',
    'right_ear',
    'mouth_left',
    'mouth_right',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist',
    'right_wrist',
    'left_pinky',
    'right_pinky',
    'left_index',
    'right_index',
    'left_thumb',
    'right_thumb',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee',
    'left_ankle',
    'right_ankle',
    'left_heel',
    'right_heel',
    'left_foot_index',
    'right_foot_index']

joint_to_plot_index = mediapipe_indices.index('nose') #You can choose what point you'd like to look at here by                                                             changing the string in the mediapipe_indices list.

In [ ]:
freemocap_3d_body_data_to_plot = freemocap_3d_body_data[:,joint_to_plot_index,:]

## 3D Plotting

We are creating a 3D plot of the skeleton movement, tracking all the joints in the `mediapipe_indices` list over time.  

The 'Play' button at the bottom allows you to watch the motion as if it were a video. Before pressing play, you can manually click and drag the plot around to orient the view of the plot.

In [ ]:
def calculate_axes_means(skeleton_3d_data):
    mx_skel = np.nanmean(skeleton_3d_data[:,0:33,0])
    my_skel = np.nanmean(skeleton_3d_data[:,0:33,1])
    mz_skel = np.nanmean(skeleton_3d_data[:,0:33,2])

    return mx_skel, my_skel, mz_skel

ax_range = 1500

mx_skel, my_skel, mz_skel = calculate_axes_means(freemocap_3d_body_data)

# Create a list of frames
frames = [go.Frame(data=[go.Scatter3d(
    x=freemocap_3d_body_data[i, :, 0],
    y=freemocap_3d_body_data[i, :, 1],
    z=freemocap_3d_body_data[i, :, 2],
    mode='markers',
    marker=dict(
        size=2,  # Adjust marker size as needed
    )
)], name=str(i)) for i in range(freemocap_3d_body_data.shape[0])]

# Define axis properties
axis = dict(
    showbackground=True,
    backgroundcolor="rgb(230, 230,230)",
    gridcolor="rgb(255, 255, 255)",
    zerolinecolor="rgb(255, 255, 255)",
)

# Create a figure
fig = go.Figure(
    data=[go.Scatter3d(
        x=freemocap_3d_body_data[0, :, 0],
        y=freemocap_3d_body_data[0, :, 1],
        z=freemocap_3d_body_data[0, :, 2],
        mode='markers',
        marker=dict(
            size=2,  # Adjust marker size as needed
        )
    )],
    layout=go.Layout(
        scene=dict(
            xaxis=dict(axis, range=[mx_skel-ax_range, mx_skel+ax_range]), # Adjust range as needed
            yaxis=dict(axis, range=[my_skel-ax_range, my_skel+ax_range]), # Adjust range as needed
            zaxis=dict(axis, range=[mz_skel-ax_range, mz_skel+ax_range]),  # Adjust range as needed
            aspectmode='cube'
        ),
        updatemenus=[dict(
            type='buttons',
            showactive=False,
            buttons=[dict(
                label='Play',
                method='animate',
                args=[None, {"frame": {"duration": 30}}]
            )]
        )]
    ),
    frames=frames
)

fig.show()

## Now lets start analysing the data!

In [ ]:

fig = make_subplots(rows=3, cols=1, subplot_titles=('Center of Mass X', 'Center of Mass Y', 'Center of Mass Z'))

fig.add_trace(
    go.Scatter(y=total_body_com_data[:, 0]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=total_body_com_data[:, 1]),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(y=total_body_com_data[:, 2]),
    row=3, col=1
)

# COM plot axes labels
fig.update_yaxes(title_text="X Axis (mm)", row=1, col=1)
fig.update_yaxes(title_text="Y Axis (mm)", row=2, col=1)
fig.update_yaxes(title_text="Z Axis (mm)", row=3, col=1)
fig.update_xaxes(title_text="Frame #", row=3, col=1)  # Assuming you only want x-axis title for the bottommost subplot

fig.update_layout(height=600, width=800, showlegend=False)
fig.show()


## Lets calculate the kinetics of the COM in the Z direction (vertical)

In [ ]:
from scipy.signal import butter, filtfilt

# First lets filter the position data to get it nice a clean
# We will use a low pass filter to smooth the data
position_com_z_data = total_body_com_data[:, 2]

# Filter Function
def butterworth_filter(data, cutoff, frame_rate, order=4, filter_type='low'):
    nyq = 0.5 * frame_rate
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype=filter_type, analog=False)

    # Adjust the padlen based on the length of the data
    padlen = min(order * 3, len(data) - 1)

    y = filtfilt(b, a, data, padlen=padlen)
    return y

# Filter the data
position_com_z_data = butterworth_filter(position_com_z_data, 5, 30, order=4, filter_type='low')


In [ ]:

velocity_com_z_data = np.diff(position_com_z_data, axis=0)
acceleration_com_z_data = np.diff(velocity_com_z_data, axis=0)

time = 1/30


In [ ]:
# Function to normalize and scale data to fit between -1 and 1
def normalize_and_scale(data):
    normalized_data = 2 * ((data - min(data)) / (max(data) - min(data))) - 1
    return normalized_data

# Apply the function to your data
scaled_position = normalize_and_scale(position_com_z_data)
scaled_velocity = normalize_and_scale(velocity_com_z_data)
scaled_acceleration = normalize_and_scale(acceleration_com_z_data)

# Plot the data
fig = make_subplots(rows=4, cols=1, subplot_titles=('Z Position', 'Z Velocity', 'Z Acceleration', 'Superimposed'))

fig.add_trace(
    go.Scatter(y=position_com_z_data, name='Z Position', line=dict(color='red')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=velocity_com_z_data, name='Z Velocity', line=dict(color='green')),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(y=acceleration_com_z_data, name='Z Acceleration', line=dict(color='blue')),
    row=3, col=1
)

# Add the scaled traces to the fourth subplot
fig.add_trace(
    go.Scatter(y=scaled_position, name='Scaled Z Position', line=dict(color='red')),
    row=4, col=1
)
fig.add_trace(
    go.Scatter(y=scaled_velocity, name='Scaled Z Velocity', line=dict(color='green')),
    row=4, col=1
)
fig.add_trace(
    go.Scatter(y=scaled_acceleration, name='Scaled Z Acceleration', line=dict(color='blue')),
    row=4, col=1
)

# COM plot axes labels
fig.update_yaxes(title_text="Z Position", row=1, col=1)
fig.update_yaxes(title_text="Z Velocity", row=2, col=1)
fig.update_yaxes(title_text="Z Acceleration", row=3, col=1)
fig.update_xaxes(title_text="Frame #", row=4, col=1)

fig.update_layout(height=800, width=800, showlegend=True)
fig.show()


## Lets look a little closer at the acceleration data.

In [ ]:
liftoff_time_stamps = []
bottom_time_stamps = []
apex_time_stamps = []

for t in range(velocity_com_z_data.size-2):
    if (velocity_com_z_data[t+2]<velocity_com_z_data[t+1]
        and velocity_com_z_data[t] < velocity_com_z_data[t+1]):
        liftoff_time_stamps.append(t+1)

for t in range(velocity_com_z_data.size-1):
    if velocity_com_z_data[t+1] > 0 and velocity_com_z_data[t] <= 0:
        bottom_time_stamps.append(t)
    if velocity_com_z_data[t+1] < 0 and velocity_com_z_data[t] >= 0:
        apex_time_stamps.append(t)

print(f"number of iterations at bottom: ", len(bottom_time_stamps))
print(f"number of iterations at apex: ", len(apex_time_stamps))

In [ ]:
fig = go.Figure()

# Add the velocity data
fig.add_trace(
    go.Scatter(y=velocity_com_z_data, mode='lines', name='Velocity COM Z')
)

fig.add_trace(
    go.Scatter(x=liftoff_time_stamps, y=velocity_com_z_data[liftoff_time_stamps],
               mode='markers', marker_symbol='star', marker_size=10,
               marker_color='yellow', name='Lift Off')
)

# Show the plot
fig.show()

In [ ]:
fig = go.Figure()

# Add the acceleration data
fig.add_trace(
    go.Scatter(y=acceleration_com_z_data, mode='lines', name='Acceleration COM Z')
)

fig.add_trace(
    go.Scatter(x=liftoff_time_stamps, y=acceleration_com_z_data[liftoff_time_stamps],
               mode='markers', marker_symbol='star', marker_size=10,
               marker_color='yellow', name='Lift Off')
)

# Show the plot
fig.show()

In [ ]:
# Create the state space plot
fig = go.Figure()

# Plot height_position vs height_velocity
fig.add_trace(
    go.Scatter(x= velocity_com_z_data, y=position_com_z_data, mode='lines+markers', marker=dict(symbol='circle', size=6, opacity=0.5), name='Data')
)

# Mark the start
fig.add_trace(
    go.Scatter(x=[velocity_com_z_data[0]], y=[position_com_z_data[0]], mode='markers', marker=dict(symbol='triangle-down', color='red', size=10), name='Start')
)

# Mark the bottom timestamps
fig.add_trace(
    go.Scatter(x=velocity_com_z_data[bottom_time_stamps], y=position_com_z_data[bottom_time_stamps], mode='markers', marker=dict(symbol='circle', color='brown', size=10), name='Bottom')
)

# Mark the apex timestamps
fig.add_trace(
    go.Scatter(x=velocity_com_z_data[apex_time_stamps], y=position_com_z_data[apex_time_stamps], mode='markers', marker=dict(symbol='circle', color='magenta', size=10), name='Apex')
)

# Mark the liftoff timestamps
fig.add_trace(
    go.Scatter(x=velocity_com_z_data[liftoff_time_stamps], y=position_com_z_data[liftoff_time_stamps], mode='markers', marker=dict(symbol='circle', color='green', size=10), name='Lift Off')
)

# Set axis labels
fig.update_layout(xaxis_title='velocity', yaxis_title='position')
fig.update_layout(height=1200, width=600, showlegend=True)

# Show the plot
fig.show()


In [ ]:
fig = go.Figure()

# Plot position_com_z_data vs velocity_com_z_data vs acceleration_com_z_data
fig.add_trace(
    go.Scatter3d(
        x=position_com_z_data,
        y=velocity_com_z_data,
        z=acceleration_com_z_data,
        mode='lines+markers',
        marker=dict(size=2, opacity=0.8),
        name='State Space Data'
    )
)

# Set axis labels
fig.update_layout(
    scene=dict(
        xaxis_title='Position COM Z [??]',
        yaxis_title='Velocity COM Z [??]',
        zaxis_title='Acceleration COM Z [??]'
    )
)

# Show the plot
fig.show()